In [12]:
!pip3 install sentence-transformers gensim SPARQLWrapper

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Vector creation pipeline

In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [4]:
#Sentences we want to encode. Example:
sentence = ['This framework generates embeddings for each input sentence']

#Sentences are encoded by calling model.encode()
embedding = model.encode(sentence)

In [5]:
embedding[0]

array([-1.76214606e-01,  1.20601386e-01, -2.93623507e-01, -2.29857996e-01,
       -8.22926089e-02,  2.37709135e-01,  3.39985043e-01, -7.80964434e-01,
        1.18127473e-01,  1.63373724e-01, -1.37715369e-01,  2.40282834e-01,
        4.25125867e-01,  1.72417581e-01,  1.05279520e-01,  5.18164277e-01,
        6.22216091e-02,  3.99286300e-01, -1.81652561e-01, -5.85578442e-01,
        4.49723750e-02, -1.72750652e-01, -2.68443674e-01, -1.47386163e-01,
       -1.89217508e-01,  1.92150250e-01, -3.83842975e-01, -3.96006763e-01,
        4.30648744e-01, -3.15320075e-01,  3.65949899e-01,  6.05159067e-02,
        3.57325613e-01,  1.59736484e-01, -3.00983757e-01,  2.63250381e-01,
       -3.94310653e-01,  1.84855431e-01, -3.99549127e-01, -2.67889589e-01,
       -5.45117438e-01, -3.13404277e-02, -4.30644095e-01,  1.33277997e-01,
       -1.74793556e-01, -4.35465306e-01, -4.77378666e-01,  7.12552369e-02,
       -7.37001821e-02,  5.69137216e-01, -2.82579482e-01,  5.24976924e-02,
       -8.20007861e-01,  

In [65]:
from SPARQLWrapper import SPARQLWrapper, JSON

NS_RESOURCE = 'http://dbpedia.org/resource/'
NS_RESOURCE_LEN = len(NS_RESOURCE)

NS_ONTOLOGY = 'http://dbpedia.org/ontology/'
NS_ONTOLOGY_LEN = len(NS_ONTOLOGY)

In [50]:
def retrieve_tbox(lang='en', offset=0):
    sparql = SPARQLWrapper('http://dbpedia.org/sparql')
    query = f"""
    SELECT ?uri ?label {{
      ?uri a ?type ; rdfs:label ?label .
      values(?type) {{ (owl:Class) (rdf:Property) }}
      filter(lang(?label) = '{lang}' && regex(?uri, "http://dbpedia.org/ontology/"))
    }} LIMIT 10000 OFFSET {offset}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    
    tbox = {}
    for result in results['results']['bindings']:
        uri = result['uri']['value']
        label = result['label']['value']
        if label not in tbox:
            tbox[label] = set()
        tbox[label].add(uri)
    return tbox

In [52]:
offset = 0
tbox = {}
while True:
    tbox_chunk = retrieve_tbox(lang='en', offset=offset)
    if len(tbox_chunk) == 0:
        break
    offset += 10000
    for k, v in tbox_chunk.items():
        if k not in tbox:
            tbox[k] = set()
        tbox[k] = tbox[k].union(v)
len(tbox)

3520

In [38]:
labels = [l.replace('\n', ' ') for l in tbox]

In [16]:
embeddings = model.encode(labels)

with open('dbpedia-ontology.vectors', 'w', encoding='utf-8') as f_out:
    f_out.write(f"{len(labels)} {len(embeddings[0])}\n")
    for label, embedding in zip(labels, embeddings):
        f_out.write(f"{label.replace(' ', '_')} {' '.join([str(x) for x in embedding])}\n")

## Query pipeline

In [8]:
from gensim.models import KeyedVectors

In [9]:
w2v_model = KeyedVectors.load_word2vec_format('dbpedia-ontology.vectors', binary=False)

In [79]:
import pandas as pd

def to_uri(label):
    # filter out glitchy Urdu URIs
    return list(filter(lambda x: 'A' <= x[NS_ONTOLOGY_LEN : NS_ONTOLOGY_LEN+1] <= 'z', tbox[label]))

def ontosim_search(term):
    result = w2v_model.most_similar(positive=model.encode([term]), topn=5)
    out = []
    for label, score in result:
        out.append({'label': label.replace('_', ' '), 'score': score})
    df = pd.DataFrame(out)
    df.insert(0, 'URIs', df['label'].map(to_uri))
    return df

In [84]:
df = ontosim_search('vice-president')
with pd.option_context('max_colwidth', None, 'display.width', 200):
    print(df)

                                                                                             URIs                label     score
0          [http://dbpedia.org/ontology/VicePresident, http://dbpedia.org/ontology/vicePresident]       vice president  0.981270
1                                                        [http://dbpedia.org/ontology/viceLeader]          vice leader  0.900534
2                  [http://dbpedia.org/ontology/President, http://dbpedia.org/ontology/president]            president  0.750471
3                                                    [http://dbpedia.org/ontology/viceChancellor]      vice chancellor  0.730483
4  [http://dbpedia.org/ontology/VicePrimeMinister, http://dbpedia.org/ontology/vicePrimeMinister]  vice prime minister  0.683350
